In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# import pybithumb
import numpy as np
import pandas as pd
from datetime import datetime
import os
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, StandardScaler
import time
# import mpl_finance as mf
from sklearn.svm import SVR
import random
import warnings
warnings.filterwarnings('ignore')

gdrive_path = '/content/gdrive/My Drive/Colab Notebooks/'

In [4]:
def low_high(reg_target, model, crop_size=0):

    none_indexs = sum(np.isnan(reg_target.values))

    if len(reg_target) - none_indexs >= crop_size:
        pass
    else:
        print('not enough data length')
        return [None] * 4

    #               for realtime svr            #
    if crop_size != 0:

        realtime_osc_reg = [np.nan] * len(reg_target)
        for i in range(none_indexs + crop_size, len(reg_target)):

            x = np.arange(len(reg_target))[i - crop_size:i].reshape(-1, 1)
            y = reg_target.values[i - crop_size:i].reshape(-1, 1)

            # print(x.shape)
            # print(y.shape)
            # print(i)

            scaler_x = StandardScaler()
            scaler_y = StandardScaler()
            x = scaler_x.fit_transform(x)
            y = scaler_y.fit_transform(y)
            model.fit(x, y)
            osc_reg = model.predict(x)
            realtime_osc_reg[i] = osc_reg[-1]

            #           SHOW CHART          #
            # span_list = list()
            # for i in range(1, len(osc_reg)):
            #     if osc_reg[i - 1] <= osc_reg[i]:
            #         span_list.append((i, i + 1))
            #
            # plt.plot(osc_reg)
            #
            # for i in range(len(span_list)):
            #     plt.axvspan(span_list[i][0], span_list[i][1], facecolor='b', alpha=0.7)
            #
            # # plt.show()
            # plt.savefig('Figure_reg/%s.png' % current_index)
            # plt.close()
            # quit()

        trade_state = [np.nan] * len(reg_target)
        for i in range(none_indexs + crop_size, len(reg_target)):
            if realtime_osc_reg[i - 1] < realtime_osc_reg[i]:
                trade_state[i] = 'up'
            else:
                trade_state[i] = 'down'

        return realtime_osc_reg, trade_state

    #           for non realtime svr        #
    else:
        x = np.arange(len(reg_target))[none_indexs:].reshape(-1, 1)
        y = reg_target.values[none_indexs:].reshape(-1, 1)

        scaler_x = StandardScaler()
        scaler_y = StandardScaler()
        x = scaler_x.fit_transform(x)
        y = scaler_y.fit_transform(y)
        model.fit(x, y)
        osc_reg = model.predict(x)
        osc_reg = [np.nan] * none_indexs + list(osc_reg)

        trade_state = [np.nan] * len(osc_reg)
        for i in range(len(osc_reg)):
            if osc_reg[i - 1] < osc_reg[i]:
                trade_state[i] = 'up'
            else:
                trade_state[i] = 'down'

        return osc_reg, trade_state

def macd(df, short=12, long=26, signal=9):

    df['MACD'] = df['close'].ewm(span=short, min_periods=short-1, adjust=False).mean() - \
        df['close'].ewm(span=long, min_periods=long-1, adjust=False).mean()
    df['MACD_Signal'] = df['MACD'].ewm(span=signal, min_periods=signal-1, adjust=False).mean()
    df['MACD_OSC'] = df.MACD - df.MACD_Signal
    df['MACD_Zero'] = 0

    return


def max_abs_scaler(x):
    scaled_x = x / abs(x).max()
    return scaled_x


def profitage(Coin, short, long, signal, model, crop_size, Date='2019-09-25', excel=0, get_fig=0):
    # try:
    #     df = pd.read_excel(dir + '%s %s ohlcv.xlsx' % (Date, Coin), index_col=0)
    #
    # except Exception as e:
    #     print('Error in loading ohlcv_data :', e)
    #     return 1.0, 1.0, 1.0
    if not Coin.endswith('.xlsx'):
        df = pybithumb.get_ohlcv(Coin, 'KRW', 'minute1')
    else:
        df = pd.read_excel(gdrive_path + './BestSet/Test_ohlc/%s' % Coin, index_col=0)

    osc_reg, trade_state = low_high(df['close'], model, crop_size)
    df = df.iloc[-len(trade_state):]

    df['osc_reg'] = osc_reg
    df['trade_state'] = trade_state
    # print(df)
    # quit()

    # 거래 수수료
    fee = 0.005

    # DatetimeIndex 를 지정해주기 번거로운 상황이기 때문에 틱을 기준으로 거래한다.
    # DatetimeIndex = df.axes[0]

    # ------------------- 상향 / 하향 매도 여부와 이익률 계산 -------------------#

    # high 가 SPP 를 건드리거나, low 가 SPM 을 건드리면 매도 체결 [ 매도 체결될 때까지 SPP 와 SPM 은 유지 !! ]
    length = len(df.index) - 1  # 데이터 갯수 = 1, m = 0  >> 데이터 갯수가 100 개면 m 번호는 99 까지 ( 1 - 100 )

    # 병합할 dataframe 초기화
    bprelay = pd.DataFrame(index=df.index, columns=['bprelay'])
    condition = pd.DataFrame(index=df.index, columns=["Condition"])
    Profits = pd.DataFrame(index=df.index, columns=["Profits"])
    # price_point = pd.DataFrame(index=np.arange(len(df)), columns=['Price_point'])

    Profits.Profits = 1.0
    Minus_Profits = 1.0

    # 오더라인과 매수가가 정해진 곳에서부터 일정시간까지 오더라인과 매수가를 만족할 때까지 대기  >> 일정시간,
    m = 0
    spanlist = []
    spanlist_limit = []
    spanlist_breakaway = []
    while m <= length:

        #               bp 찾기               #
        while True:

            #           Realtime SVR            #
            #   down >> up 이면 매수 등록    #
            if (df['trade_state'].iloc[m - 2] == 'down') and (df['trade_state'].iloc[m] == 'up') and (df['trade_state'].iloc[m - 1] == 'up'):
                # print('매수 등록')
                break

            m += 1
            if m > length:
                break

        if m > length:  # or pd.isnull(df.iloc[m]['BuyPrice']): << 이게 무슨 의미인지 모르겠음..
            break
        #       매수가는 55초 이후 up 이라고 결정된 봉의 종가    #
        bp = df['close'].iloc[m]

        #       매수 등록 완료, 매수 체결 대기     #
        start_m = m
        while True:
            bprelay["bprelay"].iloc[m] = bp

            #               매수 체결 조건 = 다음 봉의 저가가 매수가 이하이면 체결되었다고 볼 수 있다.            #
            if df.iloc[m + 1]['low'] <= bp:  # and (df.iloc[m]['high'] != df.iloc[m]['low']):  # 조건을 만족할 경우 spp 생성
                # print(df['low'].iloc[m], '<=', bp, '?')

                condition.iloc[m] = '매수 체결'
                # bprelay["bprelay"].iloc[m] = bprelay["bprelay"].iloc[m - 1]

                m += 1
                break
            else:
                m += 1
                if m > length:
                    break

                #       매수 대기 term      #
                if m - start_m > 5:
                    break

                condition.iloc[m] = '매수 대기'
                bprelay["bprelay"].iloc[m] = bprelay["bprelay"].iloc[m - 1]

        #               매수 체결, 매도 대기                #
        #           up >> down 될 때까지 (조건 만족시 매도 체결)         #
        start_m = m
        high_condition = 0
        while True:

            #      고점 매도       #
            if df['trade_state'].iloc[m] == 'down':
            # if (df['trade_state'].iloc[-2] == 'down') and (df['trade_state'].iloc[-1] == 'down'):
                # print('매도 체결')
                high_condition = 1.
                break

            condition.iloc[m] = '매도 대기'
            bprelay["bprelay"].iloc[m] = bprelay["bprelay"].iloc[m - 1]

            m += 1
            if m > length:
                break

        if m > length:
            # print(condition.iloc[m - 1])
            break

        elif high_condition == 1.:

            if np.isnan(bprelay["bprelay"].iloc[m]):
                bprelay["bprelay"].iloc[m] = bprelay["bprelay"].iloc[m - 1]

            condition.iloc[m] = "매도 체결"
            Profits.iloc[m] = df.iloc[m]['close'] / bprelay["bprelay"].iloc[m] - fee

            if float(Profits.iloc[m]) < 1:
                Minus_Profits *= float(Profits.iloc[m])
                try:
                    if start_m == m:
                        m += 1
                    spanlist.append((start_m, m))
                    spanlist_breakaway.append((start_m, m))

                except Exception as e:
                    pass

            else:
                try:
                    if start_m == m:
                        m += 1
                    spanlist.append((start_m, m))
                    spanlist_limit.append((start_m, m))

                except Exception as e:
                    pass

        # 체결시 재시작
        m += 1

    df = pd.merge(df, bprelay, how='outer', left_index=True, right_index=True)
    df = pd.merge(df, condition, how='outer', left_index=True, right_index=True)
    df = pd.merge(df, Profits, how='outer', left_index=True, right_index=True)
    df = df.reset_index(drop=True)

    if excel == 1:
        # df.to_excel("./BackTest/%s BackTest %s.xlsx" % (Date, Coin))
        df.to_excel("ExcelCheck/%s BackTest %s.xlsx" % (Date, Coin))

    profits = Profits.cumprod()  # 해당 열까지의 누적 곱!
    # print(profits)

    if np.isnan(profits.iloc[-1].item()):
        return 1.0, 1.0, 1.0, 1.0, 1.0

    # [-1] 을 사용하려면 데이터가 존재해야 되는데 데이터 전체가 결측치인 경우가 존재한다.
    if len(profits) == 0:
        return 1.0, 1.0, 1.0, 1.0, 1.0

    elif float(profits.iloc[-1]) != 1.0 and get_fig == 1:

        # 거래 체결마다 subplot 1,2 저장
        fig = plt.figure(figsize=(10, 7))
        ax = fig.add_subplot(311)
        ochl = df.iloc[:, :4]
        index = np.arange(len(ochl))
        ochl = np.hstack((np.reshape(index, (-1, 1)), ochl))
        mf.candlestick_ochl(ax, ochl, width=0.5, colorup='r', colordown='b')
        # plt.plot(df['SuperTrend'], 'blue', label='ST', linewidth=1.0)
        # plt.plot(df['SuperTrend'] + df['R2G_Gap'] * offset_low , 'c', label='ST low', linewidth=1.0)
        # plt.plot(df['SuperTrend'] + df['R2G_Gap'] * offset_high, 'm', label='ST high', linewidth=1.0)
        plt.xlim(0, len(ochl))
        plt.xticks(fontsize=10)
        plt.yticks(fontsize=10)
        plt.legend(loc='upper right', fontsize=10)
        plt.title('%.3f %.3f %.3f' % (float(profits.iloc[-1]), float(profits.iloc[-1]) / Minus_Profits, Minus_Profits))

        for trade_num in range(len(spanlist_limit)):
            plt.axvspan(spanlist_limit[trade_num][0], spanlist_limit[trade_num][1], facecolor='c', alpha=0.5)
        for trade_num in range(len(spanlist_breakaway)):
            plt.axvspan(spanlist_breakaway[trade_num][0], spanlist_breakaway[trade_num][1], facecolor='m', alpha=0.5)

        plt.subplot(312)
        plt.plot(osc_reg, 'gold', label='oscillator', linewidth=1.0)
        plt.xlim(0, len(ochl))
        # plt.title('Prefered MACD_OSC', fontsize=10)
        plt.title('Realtime close', fontsize=10)
        # plt.plot(df[['MACD_Zero']], 'g', label='zero', linewidth=1.0)
        span_list = list()
        for i in range(1, len(df)):
            if osc_reg[i - 1] < osc_reg[i]:
                span_list.append((i, i + 1))
        for i in range(len(span_list)):
            plt.axvspan(span_list[i][0], span_list[i][1], facecolor='b', alpha=0.7)

        plt.subplot(313)
        osc_reg, _ = low_high(df['close'], model)
        plt.plot(osc_reg, 'gold', label='oscillator', linewidth=1.0)
        plt.xlim(0, len(ochl))
        plt.title('Prefered close', fontsize=10)
        # plt.plot(df[['MACD_Zero']], 'g', label='zero', linewidth=1.0)
        span_list = list()
        for i in range(1, len(df)):
            if osc_reg[i - 1] < osc_reg[i]:
                span_list.append((i, i + 1))
        for i in range(len(span_list)):
            plt.axvspan(span_list[i][0], span_list[i][1], facecolor='b', alpha=0.7)

        # plot 저장 & 닫기
        # plt.show()
        plt.savefig("./Figure_reg/reg_result/%s %s.png" % (Date, Coin), dpi=300)
        plt.close()

    if profits.values[-1] != 1.:
        profits_sum = 0
        for i in range(len(Profits)):
            if Profits.values[i] != 1.:
                profits_sum += Profits.iloc[i]
        profits_avg = profits_sum / sum(Profits.values != 1.)

    else:
        profits_avg = [1.]

    # print(Profits.values.min())
    # quit()
    # print(Profits.values != 1.)
    # print(profits_avg)
    # quit()
    # print(std)

    return float(profits.iloc[-1]), float(profits.iloc[-1]) / Minus_Profits, Minus_Profits, profits_avg[
        0], Profits.values.min()


if __name__=="__main__":

    # Coinlist = pybithumb.get_tickers()
    # Fluclist = []
    # while True:
    #     try:
    #         for Coin in Coinlist:
    #             tickerinfo = pybithumb.PublicApi.ticker(Coin)
    #             data = tickerinfo['data']
    #             fluctate = data['fluctate_rate_24H']
    #             Fluclist.append(fluctate)
    #             time.sleep(1 / 90)
    #         break
    #
    #     except Exception as e:
    #         Fluclist.append(None)
    #         print('Error in making Topcoin :', e)
    #
    # Fluclist = list(map(float, Fluclist))
    # series = pd.Series(Fluclist, Coinlist)
    # series = series.sort_values(ascending=False)
    #
    # series = series[:10]
    # TopCoin = list(series.index)
    # print(TopCoin)

    # print(Date)
    # quit()
    #     # Date = '2020-02-27'

    # ohlcv_list = ['2019-10-05 LAMB ohlcv.xlsx']

    # for file in ohlcv_list:
    #     Coin = file.split()[1].split('.')[0]
    #     Date = file.split()[0]
    Date = str(datetime.now()).split()[0]
    excel_list = os.listdir(gdrive_path + './BestSet/Test_ohlc/')
    total_df = pd.DataFrame(columns=['short', 'long', 'signal', 'C', 'gamma', 'epsilon', 'total_profit_avg', 'plus_profit_avg', 'minus_profit_avg', 'avg_profit_avg', 'min_profit_avg'])
    # print(excel_list)
    # breakpoint

    short, long, signal = 105, 168, 32
    C, gamma, epsilon = 500, 20, 1
    # for short in range(35, 51, 5):
        # for long in range(short + 3, short + 100, 5):
        #     for signal in range(5, 50, 3):

    for C in [0.1, 1, 5, 10, 20, 50, 100, 200 ,500, 1000]:
      for gamma in [0.1, 1, 5, 10, 20, 50, 100, 200, 500, 1000]:
        for epsilon in [1, 0.5, 0.2, 0.1, 0.05]:

                # for Coin in TopCoin:
                #     print(short, long, signal)
                #     print(profitage(Coin, short, long, signal, Date=Date))

                #       Make folder      #
                # try:
                #     os.mkdir("./Figure_pred/%s_%s_%s/" % (short, long, signal))
                #
                # except Exception as e:
                #     print(e)
                # short, long, signal = 20, 33, 5
                model = SVR(kernel='rbf', C=C, gamma=gamma, epsilon=epsilon)


                total_profit = 0
                plus_profit = 0
                minus_profit = 0
                avg_profit = 0
                min_profit = 0
                # std = 0
                for Coin in excel_list:
                    start = time.time()
                    try:
                      result = profitage(Coin, short, long, signal, model, crop_size=300, Date=Date, get_fig=0)
                      # quit()
                      total_profit += result[0]
                      plus_profit += result[1]
                      minus_profit += result[2]
                      avg_profit += result[3]
                      min_profit += result[4]

                    except Exception as e:
                      print(e)
                      total_profit += 1.
                      plus_profit += 1.
                      minus_profit += 1.
                      avg_profit += 1.
                      min_profit += 1.

                    # std += result[5]
                total_profit_avg = total_profit / len(excel_list)
                plus_profit_avg = plus_profit / len(excel_list)
                minus_profit_avg = minus_profit / len(excel_list)
                avg_profit_avg = avg_profit / len(excel_list)
                min_profit_avg = min_profit / len(excel_list)
                # std_avg = std / len(excel_list)
                print(short, long, signal, C, gamma, epsilon, total_profit_avg, plus_profit_avg, minus_profit_avg, avg_profit_avg, min_profit_avg, '%.3f second' % (time.time() - start))

                result_df = pd.DataFrame(data=[[short, long, signal, C, gamma, epsilon, total_profit_avg, plus_profit_avg, minus_profit_avg, avg_profit_avg, min_profit_avg]],
                                         columns=['short', 'long', 'signal', 'C', 'gamma', 'epsilon', 'total_profit_avg', 'plus_profit_avg', 'minus_profit_avg', 'avg_profit_avg', 'min_profit_avg'])
                total_df = total_df.append(result_df)
                                
        total_df.to_excel(gdrive_path + './BestSet/total_df %s_%s.xlsx' % (short, C))
        # break

single positional indexer is out-of-bounds
105 168 32 0.1 0.1 1 1.0155477949498306 1.5511363647831302 0.7068354279811275 1.00378774934961 0.918098734507972 2.633 second
single positional indexer is out-of-bounds
105 168 32 0.1 0.1 0.5 0.9475343175080686 1.443686000890768 0.6688996521106787 1.0036706486403868 0.895325457445607 3.507 second
105 168 32 0.1 0.1 0.2 0.9143474616245653 1.354535824226327 0.6837462833912749 1.006843251811982 0.9219634277663543 5.400 second
single positional indexer is out-of-bounds
105 168 32 0.1 0.1 0.1 0.996332987707202 1.428976835964034 0.7065180308755015 1.0104439102126512 0.9365707549816052 6.285 second
105 168 32 0.1 0.1 0.05 1.0037310052464634 1.4507985342676264 0.6901294237742185 1.0174410761471782 0.9381195818941871 6.904 second
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
105 168 32 0.1 1 1 1.0837040573067607 1.4404998962238866 0.773910988888782 1.009947366603376 0.9406104383647312 1.684 second
105 168 32 0.1 

KeyboardInterrupt: ignored